# EE4211 Project - Question 3

Group name: WeUsOurs (Group No.26)

Group members: Liu Tianshu, Nie Hongtuo, Pan Jie, Zhang Chenxi

The whole code takes ~3 mins to finish running.

Please read our annotations while running the code:)


In Question 1, we have found out the top 5 correlated meters for each meterID. We are interesting about this results and want to explore more on this matter.

Here, in Question 3, we propose to use two of the most popular clustering algorithms: K-Means and Gaussian mixture model(GMM) to conduct the cluster for each meter. 

The clustering results will be compared with the top 5 correlated results that we obtained in Question1. We want to explore:

1,If all the top 5 correlated meters are in the same cluster (For K-Means & GMM)?

2,If some of the top 5 correlated meters are in the same cluster (For K-Means & GMM)?

3,If none of the top 5 correlated meters are in the same cluster (For K-Means & GMM)?

4,Any top 5 correlated meters exist in both K-Means cluster and GMM cluster?

5,What is the matching rate between 5 correlated meters and the clustering results (For K-Means & GMM)?

We will present detailed information in our presentation as well.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import seaborn as sns

In [2]:
df = pd.read_csv('dataport-export_gas_oct2015-mar2016.csv')
df
grouped = df.groupby(['dataid'])
len(grouped)
keys = pd.Series(df.dataid.values)
keys = keys.unique()
keys = np.sort(keys)

In [3]:
def rectify(df):
    grouped = df.groupby(['dataid'])
    for key,group in grouped:
        val = group.meter_value.values
        diff = val[1:] - val[:-1]
        defect_time = np.where(diff < 0)[0] + 1
        if len(defect_time):
            for i in range(1,len(val)):
                if val[i] > val[-1]:
                    val[i] = val[i - 1]
                elif val[i] < val[i - 1]:
                    val[i] = val[i - 1]
            df.loc[df['dataid'] == key, 'meter_value'] = val
    return df


In [4]:
df = rectify(df)
rec_df = df



In [5]:
def local2utc(time):
    return pd.Timestamp(time)
df.localminute = df.apply(lambda r: local2utc(r.localminute), axis=1)

def select_data(df, month) -> pd.DataFrame:
    column_names = df.columns.values
    values = df.values
    values = filter(lambda x: x[0].month == month, values)
    values = pd.DataFrame(values)
    for i in range(len(column_names)):
        values = values.rename(columns={i:column_names[i]})
    return values

In [6]:
def hourly_range_month(df, start_month, end_month):
    month_dict = {1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
    groups = df.groupby(['dataid'], sort=['localminute'])
    processed_group = {}
    for key in groups.groups.keys():
        new_group = []
        current_group = groups.get_group(key).values
        # init_first_point
        first_point = current_group[0]
        first_point[0] = first_point[0].replace(month = start_month, day = 1, hour = 0, minute = 0, second = 0)
        index_current_group = 0
        tmp = []
        # replace all time on the hour
        for row in current_group:
            row[0] = row[0].replace(minute = 0, second = 0)
            tmp.append(row)
        current_group = tmp
        # interpolate data 24 hour for 30 day
        for month in range(start_month, end_month + 1):
            month = month % 12 if month % 12 != 0 else 12 
            for day in range(1, month_dict[month] + 1):
                for hour in range(0, 24):
                    # make sure its the last data of the current time
                    while (True):
                        if (index_current_group >= len(current_group)):
                            break
                        current_row = current_group[index_current_group]
                        time = current_row[0]
                        if (time.month == month and time.day == day and time.hour == hour):
                            index_current_group += 1
                        else:
                            break
                    # assign data
                    current_row = current_group[index_current_group - 1]
                    time = current_row[0]
                    date = pd.Timestamp(year = time.year, month = time.month, day = time.day, hour = time.hour, minute = 0, second = 0)
                    current_row = [date, current_row[1], current_row[2]]
                    if (time.month == month and time.day == day and time.hour == hour):
                        if (len(new_group) > 0 and new_group[-1][0].month == month and new_group[-1][0].day == day and new_group[-1][0].hour == hour):
                            new_group[-1] = current_row
                        else:
                            new_group.append(current_row)
                    else:
                        date = pd.Timestamp(year = time.year, month = month, day = day, hour = hour, minute = 0, second = 0)
                        row = [date,new_group[-1][1],new_group[-1][2]]
                        new_group.append(row)
        processed_group[key] = new_group
    return processed_group

In [7]:
hourly_data_raw = hourly_range_month(df,10,15)

def hourly_data_matrix(hourly_data_raw, keys):
    hourly_data = np.zeros((157, 4368), dtype=int)
    for i in range(157):
        key = keys[i]
        monthly_value = hourly_data_raw[key]
        monthly_value = list(zip(*monthly_value))[2]
        monthly_value = np.array(monthly_value)
        hourly_data[i] = monthly_value

    return hourly_data

hourly_data = hourly_data_matrix(hourly_data_raw, keys)
hourly_data.shape
hourly_corr = np.corrcoef(hourly_data)
hourly_corr.shape
import datetime
df = pd.read_csv('dataport-export_gas_oct2015-mar2016.csv')
def rectify(df):
    
    grouped = df.groupby(['dataid'])
    for key,group in grouped:
        val = group.meter_value.values
        diff = val[1:] - val[:-1]
        defect_time = np.where(diff < 0)[0] + 1
        if len(defect_time):
            for i in range(1,len(val)):
                if val[i] > val[-1]:
                    val[i] = val[i - 1]
                elif val[i] < val[i - 1]:
                    val[i] = val[i - 1]
            df.loc[df['dataid'] == key, 'meter_value'] = val
    return df

First,similar as Q1.3, we find out the top 5 relative families for each MeterID. 

In [8]:
Top5 = []
for i in range(157):
    order = np.argsort(hourly_corr[i,:])
    #print('The top 5 relative families of consumer No.', keys[i], 'are:\t', keys[np.flip(order)[1:6]])
    top5 = [keys[i],keys[np.flip(order)[1:6]]]
    Top5.append(top5)
    #print(Top5)
    print('The top 5 relative families of consumer No.', keys[i], 'are:\t', keys[np.flip(order)[1:6]])

The top 5 relative families of consumer No. 35 are:	 [2449 1697 5810 7287 5403]
The top 5 relative families of consumer No. 44 are:	 [1697 1283   35 2449  484]
The top 5 relative families of consumer No. 77 are:	 [9849 1697   35   44 2449]
The top 5 relative families of consumer No. 94 are:	 [2575 5892 9134 5129 1507]
The top 5 relative families of consumer No. 114 are:	 [1556 4514 2094 3893 1619]
The top 5 relative families of consumer No. 187 are:	 [9766 4296 8086 1507 2018]
The top 5 relative families of consumer No. 222 are:	 [7965   44 1283 2965   77]
The top 5 relative families of consumer No. 252 are:	 [3577 5395 4732 1556 5814]
The top 5 relative families of consumer No. 370 are:	 [7117 1589 3527 6836 1714]
The top 5 relative families of consumer No. 483 are:	 [3367 4193 5439 8829 7017]
The top 5 relative families of consumer No. 484 are:	 [5484 5403 5892 5810 9982]
The top 5 relative families of consumer No. 661 are:	 [7794 5814 3723  739 4514]
The top 5 relative families of c

For a better performance, here we use the hourly useage for each meter.

In [9]:
month = 10

In [10]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
if month >= 10:
    start_hour = (month - 10)*728
    end_hour = start_hour + 728 - 1
if month <10:
    start_hour = (month + 12 - 10)*728
    end_hour = start_hour + 728 - 1


hourly_data = hourly_data[:,start_hour:end_hour]

meter_num,hour_num = hourly_data.shape
hourly_usage = np.zeros(shape=(meter_num,hour_num))

for i in range(157):
    hourly_usage[i] = hourly_data[i] - hourly_data[i][0]   


Let start from K-means.
By default, we set the number of clusters as 4.

In [11]:
Clusters = 4

kmeans = KMeans(n_clusters=Clusters)
kmeans.fit(hourly_usage)
cluster = kmeans.predict(hourly_usage)
cluster = np.c_[cluster,keys]
Clus = []

for i in range(Clusters):
    idx = np.where(cluster[:,0] == i)
    l = cluster[idx[0]][:,1]
    Clus.append(list(l))
 
print(Clus)
YK = []
Not_match_num = 0
for i in range(len(Top5)):
    print('For meted id '+str(Top5[i][0])+':')
    Idx =  np.where(cluster[:,1] == Top5[i][0])
    label = cluster[Idx,0]
    No = []
    yk = []
    for j in Top5[i][1]:
        if j not in Clus[label[0][0]]:
            No.append(j)
            Not_match_num = Not_match_num + 1
        else:
            yk.append(j)
    YK.append(yk)
    if No == []:
        print('All its top 5 correlated meters are in the same clusters')
    else:
        print('The meter(s) is(are) of top 5 correlated but not the same cluster:')
        print(No)
    print('------------------------')


print('#################')
print('#################')
print('#################')
print(str(round(100-100*Not_match_num/(157*5),3))+
      '% of the K-means result can match with top 5 correlation')

[[35, 44, 77, 94, 114, 252, 483, 484, 661, 739, 744, 1185, 1556, 1589, 1619, 1714, 1791, 1792, 1800, 1801, 2034, 2072, 2094, 2233, 2335, 2378, 2470, 2575, 2638, 2645, 2980, 3039, 3134, 3367, 3527, 3577, 3778, 3849, 3893, 4031, 4193, 4296, 4352, 4356, 4373, 4421, 4447, 4732, 4767, 4998, 5129, 5131, 5275, 5403, 5439, 5636, 5785, 5814, 6412, 6578, 6830, 6836, 6863, 7017, 7429, 7741, 7900, 7965, 8084, 8155, 8156, 8467, 8703, 8829, 8890, 9052, 9134, 9278, 9631, 9639, 9729, 9849, 9956, 9982], [222], [187, 1042, 1086, 1283, 1415, 1507, 1697, 1718, 1790, 2018, 2129, 2449, 3723, 4029, 4514, 5193, 5484, 5810, 5892, 5972, 6910, 7030, 7287, 7460, 7674, 7682, 7794, 7919, 8086, 9121, 9295, 9474, 9766], [370, 871, 1103, 1403, 2461, 2755, 2814, 2818, 2945, 2946, 2965, 3036, 3310, 3544, 3635, 3918, 4228, 4671, 4874, 5317, 5395, 5545, 5658, 6101, 6505, 6673, 6685, 7016, 7117, 7566, 7739, 7989, 8059, 8244, 8386, 8967, 9160, 9600, 9620]]
For meted id 35:
The meter(s) is(are) of top 5 correlated but not th

From the result we can tell that:
1,For most of the meters, some of its top 5 correlated meters are in the K-Means cluster.
2,For several meters,all of its top 5 correlated meters are in the K-Means cluster.
3,For only a small amount of meters, none of its top 5 correlated meters are in the K-Means cluster.


Now let's try another method, GMM

In [12]:
GMM = GaussianMixture(n_components=Clusters,covariance_type='tied',reg_covar=0.1)
GMM.fit(hourly_usage)
cluster = GMM.predict(hourly_usage)
cluster = np.c_[cluster,keys]
Clus = []

for i in range(Clusters):
    idx = np.where(cluster[:,0] == i)
    l = cluster[idx[0]][:,1]
    Clus.append(list(l))
 
print(Clus)
YG = []
Not_match_num = 0
for i in range(len(Top5)):
    print('For meted id '+str(Top5[i][0])+':')
    Idx =  np.where(cluster[:,1] == Top5[i][0])
    label = cluster[Idx,0]
    No = []
    yg = []
    for j in Top5[i][1]:
        if j not in Clus[label[0][0]]:
            No.append(j)
            Not_match_num = Not_match_num + 1
        else:
            yg.append(j)
    YG.append(yg)
    if No == []:
        print('All its top 5 correlated meters are in the same clusters')
    else:
        print('The meter(s) is(are) of top 5 correlated but not the same cluster:')
        print(No)
    print('------------------------')


print('#################')
print('#################')
print('#################')
print(str(round(100-100*Not_match_num/(157*5),3))+
      '% of the GMM result can match with top 5 correlation')


[[187, 484, 1042, 1086, 1283, 1415, 1507, 1697, 1718, 1790, 1800, 2018, 2094, 2129, 2449, 3723, 4029, 4296, 4356, 4514, 5193, 5403, 5484, 5636, 5810, 5892, 5972, 6910, 7030, 7287, 7460, 7674, 7682, 7794, 7919, 8086, 9121, 9295, 9474, 9766, 9982], [222], [35, 44, 77, 94, 114, 252, 483, 661, 739, 744, 1185, 1556, 1589, 1619, 1714, 1791, 1792, 1801, 2034, 2072, 2233, 2335, 2378, 2461, 2470, 2575, 2638, 2645, 2980, 3039, 3134, 3367, 3527, 3577, 3778, 3849, 3893, 4031, 4193, 4228, 4352, 4373, 4421, 4447, 4732, 4767, 4998, 5129, 5131, 5275, 5395, 5439, 5785, 5814, 6412, 6578, 6830, 6836, 6863, 7017, 7117, 7429, 7741, 7900, 7965, 8084, 8155, 8156, 8467, 8703, 8829, 8890, 9052, 9134, 9278, 9631, 9639, 9729, 9849, 9956], [370, 871, 1103, 1403, 2755, 2814, 2818, 2945, 2946, 2965, 3036, 3310, 3544, 3635, 3918, 4671, 4874, 5317, 5545, 5658, 6101, 6505, 6673, 6685, 7016, 7566, 7739, 7989, 8059, 8244, 8386, 8967, 9160, 9600, 9620]]
For meted id 35:
The meter(s) is(are) of top 5 correlated but not th

From the result we can tell that:
1,For most of the meters, some of its top 5 correlated meters are in the GMM cluster.
2,For several meters,all of its top 5 correlated meters are in the GMM cluster.
3,For only a small amount of meters, none of its top 5 correlated meters are in the GMM cluster.

Now let's see which top 5 correlated meter(s) is(are) in the both K-means cluster and GMM cluster.

In [13]:
The_same = []
for i in range(len(YG)):
    l = []
    l.append(cluster[i,1])
    if YG[i] != []:
        if YK[i]!= []:
            the_same = [x for x in YG[i] if x in YK[i]]
            if the_same != []:
                print('Based on correlation,K-means and GMM')
                print('MeterID:'+ str(cluster[i,1]))
                print('and')
                print(the_same)
                print('are in the both K-Means and GMM clusters')
                print(' ')
                l.append(the_same)
    The_same.append(l)
    
for i in range(157):
    if len(The_same[i]) == 1:
        The_same[i].append([])


TOP5 = []
ALL = []
for i in range(157):
    TOP5.append(Top5[i][1])
    ALL.append(The_same[i][1])

all_match = 0
for i in range(157):
    all_match = all_match + len(ALL[i])


print(str(round(100*Not_match_num/(157*5),3))+
      '% result of these 3 methods can match up')

Based on correlation,K-means and GMM
MeterID:44
and
[35]
are in the both K-Means and GMM clusters
 
Based on correlation,K-means and GMM
MeterID:77
and
[9849, 35, 44]
are in the both K-Means and GMM clusters
 
Based on correlation,K-means and GMM
MeterID:94
and
[2575, 9134, 5129]
are in the both K-Means and GMM clusters
 
Based on correlation,K-means and GMM
MeterID:114
and
[1556, 3893, 1619]
are in the both K-Means and GMM clusters
 
Based on correlation,K-means and GMM
MeterID:187
and
[9766, 8086, 1507, 2018]
are in the both K-Means and GMM clusters
 
Based on correlation,K-means and GMM
MeterID:252
and
[3577, 4732, 1556, 5814]
are in the both K-Means and GMM clusters
 
Based on correlation,K-means and GMM
MeterID:483
and
[3367, 4193, 5439, 8829, 7017]
are in the both K-Means and GMM clusters
 
Based on correlation,K-means and GMM
MeterID:484
and
[5403, 9982]
are in the both K-Means and GMM clusters
 
Based on correlation,K-means and GMM
MeterID:661
and
[5814, 739]
are in the both K-

We can tell that even we restrict the measurements, the meter can still match a lot.You can also see the summary table below  

In [14]:
meter_close = {'meterid':keys,'TOP5':TOP5,'K-means':YK,'GMM':YG,'all':ALL}
meter_close = pd.DataFrame(meter_close)

print(meter_close)


     meterid                            TOP5                   K-means  \
0         35  [2449, 1697, 5810, 7287, 5403]                    [5403]   
1         44     [1697, 1283, 35, 2449, 484]                 [35, 484]   
2         77      [9849, 1697, 35, 44, 2449]            [9849, 35, 44]   
3         94  [2575, 5892, 9134, 5129, 1507]        [2575, 9134, 5129]   
4        114  [1556, 4514, 2094, 3893, 1619]  [1556, 2094, 3893, 1619]   
..       ...                             ...                       ...   
152     9729  [4031, 2034, 3544, 1801, 8829]  [4031, 2034, 1801, 8829]   
153     9766   [4296, 8086, 2018, 1801, 187]         [8086, 2018, 187]   
154     9849        [77, 35, 1697, 2449, 44]              [77, 35, 44]   
155     9956   [7429, 2945, 7900, 744, 5131]   [7429, 7900, 744, 5131]   
156     9982   [5403, 484, 4296, 1042, 9766]         [5403, 484, 4296]   

                               GMM                       all  
0                               []              